In [ ]:
# xmlrpc_server.ipynb
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client as xmlrpclib
import datetime
import pandas as pd
import numpy as np
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8009), 
                            requestHandler=RequestHandler)

try:
    stats_server = xmlrpclib.ServerProxy("http://localhost:8018")
except xmlrpclib.Error:
    print("Сервер с портом 8018 не доступен")


# Добавление в лог через сервер
def add_log(log_line):
    try:
        stats_server.add_log(log_line)
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    return True
    
# Тест
def geting():
    try:
        return stats_server.get_log()
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
server.register_function(geting, 'geting')

# Тест
def ping():
    try:
        add_log('Ping')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    try:
        add_log('Now')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    try:
        add_log('Type')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    try:
        add_log('Sum')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    try:
        add_log('Pow')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname, sbirth):
    try:
        add_log('Badboys')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    frame = pd.read_csv('badboys/bad_boys2.csv', header=0, sep=',', encoding='utf8')
    
    exist = any((frame['Surname'] == sname) & (frame['Birth'] == sbirth))
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    try:
        add_log('Bin')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    data = bin_data.data
    return xmlrpclib.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

def binary_image(bin_data, threshold):
    try:
        add_log('Bin')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    img_arr = pickle.loads(bin_data.data)
    
    # Преобразование изображения в оттенки серого
    img_gray = np.mean(img_arr, axis=2)
    
    # Бинаризация изображения
    binary_img = np.where(img_gray > threshold, 255, 0).astype(np.uint8)
    
    # Преобразование обратно в RGB
    img_bin = np.stack([binary_img, binary_img, binary_img], axis=2)
    
    pimg = pickle.dumps(img_bin)
    return xmlrpclib.Binary(pimg)
server.register_function(binary_image, 'binary_image')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    try:
        add_log('Inversion')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    img_arr = pickle.loads(bin_data.data)
    
    if len(img_arr.shape) == 3:
        height = img_arr.shape[0]
        width = img_arr.shape[1]
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 2
                55 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
    elif len(img_arr.shape) == 2:
        height = img_arr.shape[0]
        width = img_arr.shape[1]    
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    else:
        raise ValueError("Invalid image array shape!")

    pimg = pickle.dumps(img_arr)        
    return xmlrpclib.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

def send_back_flip(bin_data):
    try:
        add_log('Flip')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    
    img_arr = pickle.loads(bin_data.data)
    
    if len(img_arr.shape) == 3:
        height = img_arr.shape[0]
        width = img_arr.shape[1]    
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
    elif len(img_arr.shape) == 2:
        height = img_arr.shape[0]
        width = img_arr.shape[1]    
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    else:
        raise ValueError("Invalid image array shape!")

    pimg = pickle.dumps(img_arr)        
    return xmlrpclib.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

def flip_image(data_arg):
    try:
        add_log('Flip')
    except ConnectionRefusedError:
        print('Сервер недоступен')
        
    picture_matrix = pickle.loads(data_arg.data)
    height = picture_matrix.shape[0]
    width = picture_matrix.shape[1] - 1

    for i in range(height):
        for j in range(width // 2):
            original = picture_matrix[i][j].copy()
            picture_matrix[i][j] = picture_matrix[i][width - j].copy()
            picture_matrix[i][width - j] = original
    result = pickle.dumps(picture_matrix)

    return xmlrpclib.Binary(result)
server.register_function(flip_image, 'flip_image')

print("Listening on port 8009...")
server.serve_forever()

Listening on port 8009...
